### 문제점 목록
- 문제점1. 의미없는 문장을 함수로 제거하는데 한계가 있음 -> 6번 과정에서 길이가 짧은 Description 출력하여 하나씩 확인하고 인덱스 제거
- 문제점2. 1의 과정을 거쳐도 여전히 조회되지 않는 행이 있음..  ex. museum_data_6_final.csv 파일의 행 410, 431 등
- 문제점3. 같은 Title + 다른 Description의 경우, Title에 임의로 넘버링 해주기로 하였지만 하나씩 찾아보기 어려움
- 문제점4. 특수문자를 전부 삭제하여 문장의 원래 의미를 알기 힘들게 됨... ex. "조속송민고필 화조도 8폭 병풍,조속은 광해군 재위 **16081623** 때에" 에서 숫자가 연도로 표시되어야 하지만 특수문자 삭제로 숫자만 남았음
- 문제점5. <<문제점4>>를 해결하기 위해 컬럼을 나누어 제거하는 문자를 다르게 적용해 봤지만 전처리가 잘 안됨. 왜???????
    - 예시: 

            # Title 컬럼 전처리 함수
            def clean_title(text):
                if isinstance(text, str):
                    text = re.sub(r'[^0-9가-힣\s:/._?&=-~\'()\[\]《》「」,·『』]', '', text)  # 한자, 영어 제거 
                    text = re.sub(r'\(\s*\)', '', text)  # 빈 괄호 제거
                    text = re.sub(r'"', '', text)  # 큰따옴표 제거
                    text = text.replace("...", "")  # "..." 제거
                    text = re.sub(r'\s+', ' ', text).strip()  # 중복 공백 제거
                return text

            # Description 컬럼 전처리 함수
            def clean_description(text):
                if isinstance(text, str):
                    text = re.sub(r'[^A-Za-z0-9가-힣\s:/._?&=-~\"\'()\[\]《》「」,·『』]', '', text)  # 한자 제거
                    text = re.sub(r'\(\s*\)', '', text)  # 빈 () 제거
                    text = re.sub(r'『\s*』', '', text)  # 빈 『』 제거
                    text = re.sub(r'「\s*」', '', text)  # 빈 「」 제거
                    text = re.sub(r'\[\s*\]', '', text)  # 빈 [] 제거
                    text = re.sub(r'《\s*》', '', text)  # 빈 《》 제거
                    text = re.sub(r'^\?$', '', text)  # 단독 '?' 제거
                    text = re.sub(r'\s+', ' ', text).strip()  # 중복 공백 제거
                return text
            

### 진행 과정
1. 파일 로드 및 결측치 제거
2. Image URL 컬럼 삭제(선택 사항) 
3. 전처리 함수: 한자 제거
4. 3번의 과정으로 생긴 결측치 제거
5. 의미없는 문장 제거 함수 / 인덱스 정렬
6. Description 컬럼의 의미없는 문장 인덱스 제거(수작업)
7. 파일 저장

### 1. 파일 로드 및 결측치 제거

In [6]:
import pandas as pd
import re

# # ✅ 1️⃣ CSV 파일 불러오기 / 결측치 제거
# file_path = "museum_data_6.csv"  # 파일 경로 설정
# df = pd.read_csv(file_path)
# df = df.dropna()
# df.info()

In [14]:
import pandas as pd
import glob

# ✅ 1️⃣ CSV 파일 경로 리스트
file_paths = ["museum_data_1.csv", "museum_data_2.csv", "museum_data_3.csv", "museum_data_4.csv", "museum_data_5.csv", "museum_data_324.csv", "museum_data_35640.csv"]

# ✅ 2️⃣ 모든 CSV 파일 불러와 병합
df_list = [pd.read_csv(file) for file in file_paths]
df_merged = pd.concat(df_list, ignore_index=True)
print(len(df_merged))

# ✅ 3️⃣ 중복 행 제거 (유물 제목 기준으로 중복 제거)
df_merged = df_merged.drop_duplicates(subset=["Title"])
print(len(df_merged))

# ✅ 4️⃣ 결측치 확인 및 처리
df_merged = df_merged.dropna(subset=["Title", "Description"])  # Title, Description이 없는 행 제거
print(len(df_merged))

print(f"✅ 병합된 데이터 개수: {len(df_merged)}")


# ✅ 5️⃣ 병합된 데이터 CSV 파일로 저장
df_merged.to_csv("merged_museum_data.csv", index=False)

36012
18334
7751
✅ 병합된 데이터 개수: 7751


### 2. Image URL 컬럼 삭제(선택 사항) 

In [ ]:
# # Image URL 컬럼 삭제(선택 사항)
# df = df.drop(columns=["Image URL"])
# df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 567 entries, 1 to 3560
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        567 non-null    object
 1   Description  567 non-null    object
dtypes: object(2)
memory usage: 13.3+ KB


### 3. 전처리 함수: 한자 제거

In [15]:
import re

# ✅ 1️⃣ 한국어 이외 모든 문자 삭제 (영어, 특수문자, 한자 포함)
def keep_korean_only(text):
    if isinstance(text, str):
        text = re.sub(r'[^가-힣0-9\s]', '', text)  # 한국어와 숫자만 남기기
        text = re.sub(r'\s+', ' ', text).strip()  # 중복 공백 제거 및 앞뒤 공백 정리
    return text

# ✅ 2️⃣ 특정 컬럼에 적용 (Title, Description)
df_merged["Title"] = df_merged["Title"].apply(keep_korean_only)
df_merged["Description"] = df_merged["Description"].apply(keep_korean_only)

# ✅ 3️⃣ 데이터 정보 확인
df_merged.info()

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)


print("✅ 한국어만 남기고 전처리 완료!")



<class 'pandas.core.frame.DataFrame'>
Index: 7751 entries, 6 to 36011
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        7751 non-null   object
 1   Image URL    7751 non-null   object
 2   Description  7751 non-null   object
dtypes: object(3)
memory usage: 242.2+ KB
✅ 한국어만 남기고 전처리 완료!


In [ ]:
# def clean_text(text):
#     if isinstance(text, str):
#         # ✅ 특수문자는 유지, 영어, 숫자, 한글 외 다른 문자 제거
#         text = re.sub(r'[^A-Za-z0-9가-힣\s:/._?&=-]', '', text)  
        
#         # ✅ 중복 공백 제거 및 앞뒤 공백 정리
#         text = re.sub(r'\s+', ' ', text).strip()
    
#     return text

# df_merged = df_merged.map(clean_text)
# df_merged.info()

# cleaned_file_path = "merged_museum_data.csv"
# df_cleaned.to_csv(cleaned_file_path, index=False)

# print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")



<class 'pandas.core.frame.DataFrame'>
Index: 5378 entries, 6 to 24003
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        5378 non-null   object
 1   Image URL    5378 non-null   object
 2   Description  5378 non-null   object
dtypes: object(3)
memory usage: 168.1+ KB


### 4. 3번 과정에서 생긴 결측치 제거

In [16]:
import pandas as pd
import numpy as np

# ✅ 1️⃣ CSV 파일 불러오기
file_path = "merged_museum_data.csv"  # 정제된 파일 경로
df_merged = pd.read_csv(file_path)  # CSV 파일 읽기

# ✅ 2️⃣ Title & Description 컬럼의 공백을 NaN 값으로 변환
df_merged[["Title", "Description"]] = df_merged[["Title", "Description"]].replace(r'^\s*$', np.nan, regex=True)  

# ✅ 3️⃣ NaN 값이 포함된 행 삭제
df_merged = df_merged.dropna(subset=["Title", "Description"])
df_merged.info()

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)

print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")



<class 'pandas.core.frame.DataFrame'>
Index: 7619 entries, 0 to 7750
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        7619 non-null   object
 1   Image URL    7619 non-null   object
 2   Description  7619 non-null   object
dtypes: object(3)
memory usage: 238.1+ KB
🔹 파일이 저장되었습니다: merged_museum_data.csv


### 5. 의미없는 문장 제거 함수 / 인덱스 정렬

In [17]:
# ✅ 2️⃣ 의미 없는 문장(불완전한 문장) 제거 함수
def remove_incomplete_sentences(text):
    if isinstance(text, str):
        # ✅ 조사, 접속사만 있는 불완전한 문장 패턴 (공백 포함)
        meaningless_patterns = r'^\s*(달고|달았음|으로|로서|이고|한|있음|만|마다|새긴|달었음|록로|팠음|폭|은|에는||을|를|에|과|와|및|의|로|에서|과 함께|의 경우|을 위해|을 통해|및 에선|을 하였음|에 있음|을 들었음|으로 하였음|및 으로|을 하여|있음|달았음|하였음)\s*$'
        
        # ✅ 문장 전체가 패턴과 일치하는 경우 NaN으로 변환
        if re.fullmatch(meaningless_patterns, text):
            return np.nan  # NaN으로 변환하여 삭제할 수 있도록 처리
        
    return text  # 정상적인 문장은 그대로 반환

# ✅ 3️⃣ Description 컬럼에서 의미 없는 문장 제거
df_merged["Description"] = df_merged["Description"].apply(remove_incomplete_sentences)
print(len(df_merged))

# ✅ 4️⃣ NaN 값이 포함된 행 삭제
df_merged = df_merged.dropna(subset=["Description"])

# ✅ 5️⃣ 인덱스 재정렬
df_merged = df_merged.reset_index(drop=True)
print(len(df_merged))

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)

print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")

7619
7615
🔹 파일이 저장되었습니다: merged_museum_data.csv


### 6. Description 컬럼의 의미없는 문장 인덱스 제거(수작업)

In [18]:
# ✅ Description 컬럼의 문자열이 일정 길이 이하인 행 필터링
df_short_desc = df_merged[df_merged["Description"].str.len() <= 20]

print(f"🔹 총 {len(df_short_desc)}개의 행이 필터링되었습니다.")
print(df_short_desc['Description'].head(20))


🔹 총 147개의 행이 필터링되었습니다.
25       용의 정수리에 최자가 새겨져 있다
26                 왕 명문이 있다
49              손잡이에 초문이 있다
62           원판번호 66319 청동추
69      측면에는 장사군전 글자가 양각되었다
87        원판번호 61914 나전길상문함
233           원판번호 70610 장도
270          원판번호 290032 한경
273    원판번호 97113 격납상자제목 종루
329            원판번호 7656 문서
347       앞면은 상평오 글자가 찍혀 있다
368    원판번호 300113 도서자료 지석묘
372                새겨진 문양 등
382        원판번호 7209 통도사탑비명
398      원판번호 8564 격납상자제목 병
467            바닥에 정이 음각되었다
491       내면 가운데 장흥고를 인화하였다
505            원판번호 6475 서간
534     외면에 려 모 을 등의 문자가 있다
576         원판번호 6207 귀형화약통
Name: Description, dtype: object


### 7. 6번에서 거른 문장 삭제 후 파일 저장

In [ ]:
# df_short_desc = df_short_desc.drop(index=[29, 83, 157, 180, 190, 192, 238, 256, 275, 280, 281, 286, 318, 331, 360, 395, 396, 397, 408, 429])
# df_short_desc = df_merged.reset_index(drop=True)  # 🔹 인덱스 재정렬 (기존 인덱스 삭제)

# cleaned_file_path = "museum_data_6_final.csv"
# df_short_desc.to_csv(cleaned_file_path, index=False)

# print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")


🔹 파일이 저장되었습니다: museum_data_6_final.csv
